In [1]:
!pip install pandas
!pip install pybboxes

In [ ]:
!wget https://he-public-data.s3.ap-southeast-1.amazonaws.com/dataset.zip
!unzip dataset
!mv dataset smartathone-dataset

In [40]:
import pybboxes as pbx
import pandas as pd
import shutil
import os
import filecmp

In [66]:
new_dataset_path = "../datasets/visual-pollution"
old_dataset_path = "../smartathone-dataset"

In [67]:
def seprate_train_images_from_test_image(data_type='train'):
    images=f'{old_dataset_path}/images/'

    images_paths=pd.read_csv(f'{old_dataset_path}/{data_type}.csv')

    os.makedirs(f'../datasets/visual-pollution/{data_type}/images')
    test_data= pd.DataFrame()
    for filename in os.listdir(images):
        if(images_paths["image_path"].eq(filename).any()) == True:
            new_path = new_dataset_path+'/'+data_type+'/images/'+filename
#             shutil.move(f'{old_dataset_path}/images/{filename}',new_path)
            shutil.copy(f'{old_dataset_path}/images/{filename}',new_path)

In [55]:
seprate_train_images_from_test_image(data_type='train')
shutil.copy(f'{old_dataset_path}/train.csv',new_dataset_path+"/train")

In [56]:
seprate_train_images_from_test_image(data_type='test')

In [68]:
df = pd.read_csv(f'{new_dataset_path}/train/train.csv')  
df

,class,image_path,name,xmax,xmin,ymax,ymin
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797.0,701.0,262.0,211.0
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932.0,786.0,329.0,238.0
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736.0,657.0,275.0,229.0
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986.0,786.0,136.0,0.0
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667.0,549.0,228.0,179.0
...,...,...,...,...,...,...,...
19945,4.0,081e7bb3832ec5bb25276db161a96274.jpg,CONSTRUCTION_ROAD,1025.0,600.0,408.0,148.0
19946,2.0,1ff38a7af7f13b1201d17c6e1829373a.jpg,POTHOLES,657.0,418.0,364.0,282.0
19947,2.0,1ff38a7af7f13b1201d17c6e1829373a.jpg,POTHOLES,507.0,338.0,436.0,283.0
19948,7.0,ac97490f13140fc1bfe613ec69301b34.jpg,BAD_BILLBOARD,956.0,713.0,110.0,25.0


In [69]:
df["class"].value_counts()

3.0     8597
4.0     2730
2.0     2625
9.0     2253
7.0     1555
0.0     1124
8.0      748
10.0     127
1.0      107
5.0       83
6.0        1
Name: class, dtype: int64

In [70]:
df.loc[df['class'] == 6.0]

,class,image_path,name,xmax,xmin,ymax,ymin
19353,6.0,53d3797457a0d2e3afe146e2f797e77e.jpg,BAD_STREETLIGHT,985.0,359.0,161.0,-47.0


In [71]:
df["name"].value_counts()

GARBAGE              8597
CONSTRUCTION_ROAD    2730
POTHOLES             2625
CLUTTER_SIDEWALK     2253
BAD_BILLBOARD        1555
GRAFFITI             1124
SAND_ON_ROAD          748
UNKEPT_FACADE         127
FADED_SIGNAGE         107
BROKEN_SIGNAGE         83
BAD_STREETLIGHT         1
Name: name, dtype: int64

In [72]:
df["image_path"].value_counts()

291d3a6d38674da0e4b05135008bceb1.jpg    22
8b3335738dc291770ffb3613810128f4.jpg    19
fec90b848c9b6c4bcc1ec562597c55d8.jpg    19
0a2bc0dc2371794509f4b776aff0dd88.jpg    19
270c3a56f6a9abef21d61aec1eb53e88.jpg    19
                                        ..
1596246bbf778e2120656ca0eb5c2272.jpg     1
1726f638d4df5e9140ee7a691d155220.jpg     1
fcdf201c3cbe3543809d35eed38e2e40.jpg     1
320c8cc65946b84f11348cfb24496fd8.jpg     1
4688a6fd763942a56f0ae6510393a932.jpg     1
Name: image_path, Length: 7874, dtype: int64

In [73]:
def resize_bounding_box(x_min, y_min, x_max, y_max):
    yolo_bbox = (abs(x_min*2), abs(y_min*2), abs(x_max*2), abs(y_max*2))
    r = yolo_bbox
    r = ' '.join(map(str,r))

    return r

In [76]:
i=0
j=1
os.makedirs(f"{new_dataset_path}/batches/batch-{j}/labels/")
os.makedirs(f"{new_dataset_path}/batches/batch-{j}/images/")
  
for index, row in df.iterrows():
    i = i+1
    if(i < 2000):

        with open(f'{new_dataset_path}/batches/batch-{j}/labels/{row["image_path"]}.txt',"a") as file:
            file.write(str(row["name"])+" "+ resize_bounding_box(row["xmin"], row["ymin"], row["xmax"], row["ymax"]) + " \n")
        shutil.copy(f'{old_dataset_path}/images/{row["image_path"]}', f"{new_dataset_path}/batches/batch-{j}/images/") 
    else:
        i=0
        j= j+1
        os.makedirs(f"{new_dataset_path}/batches/batch-{j}/labels/")
        os.makedirs(f"{new_dataset_path}/batches/batch-{j}/images/")
      